In [ ]:
from pyspark import SparkContext


In [ ]:
sc = SparkContext("local", "APP")

transactions_data = [
    "U101,P301,250.5,2023-09-12 14:05:21",
    "U102,P302,150.0,2023-09-13 11:15:30",
    "U101,P303,320.0,2023-10-01 16:45:00",
    "U103,P301,500.0,2023-09-20 10:00:00",
    "U104,P304,75.0,2023-09-22 12:30:00",
    "U105,P305,600.0,2023-10-05 14:25:00",
    "U102,P301,80.0,2023-10-06 09:00:00",
    "U106,P306,400.0,2023-09-12 18:10:00",
    "U101,P302,50.0,2023-10-12 19:30:00",
    "U107,P307,350.0,2023-09-30 20:00:00",
    "U105,P303,275.0,2023-10-02 11:45:00"
]

t_rdd = sc.parallelize(transactions_data, 3)

users_data = [
    "U101,John,28,New York",
    "U102,Alice,34,Chicago",
    "U103,David,22,Los Angeles",
    "U104,Eva,29,Chicago",
    "U105,Mark,30,New York",
    "U106,Sophie,26,San Francisco",
    "U107,Liam,32,Chicago",
    "U108,Noah,25,Miami"
]

u_rdd = sc.parallelize(users_data, 3)

p_rdd = sc.textFile(r'C:\Users\rahul\OneDrive\Documents\products.txt')


In [ ]:
# 1️⃣ Read transactions.txt and calculate the total amount spent by each user.

m_1 = t_rdd.map(lambda x: x.split(',')).map(lambda y: (y[0], int(float(y[2]))))
result_1 = m_1.reduceByKey(lambda x, y: x + y)
print(result_1.collect())

In [ ]:
# 2️⃣ Find the top 5 highest-spending users based on total amount spent from transactions.txt.

print(result_1.takeOrdered(5,key = lambda x : -x[1]))

In [ ]:
# 3️⃣ From users.txt, count the number of users in each city.

m_3 = u_rdd.map(lambda x: x.split(",")).map(lambda g: (g[3], 1))
result_3 = m_3.reduceByKey(lambda f, x: f + x).sortBy(lambda l: l[1])
print(result_3.collect())


In [ ]:
# 4️⃣ Join transactions.txt with users.txt to display: user_id, name, total amount spent

def parsing(nums):
    sp = nums.split(",")
    return (sp[0], int(float(sp[2])))

def parsing1(nums):
    sp = nums.split(",")
    return (sp[0], sp[1])

parse_t = t_rdd.map(parsing)
parse_u = u_rdd.map(parsing1)
total = parse_t.reduceByKey(lambda x, y: x + y)
jn = parse_u.join(total)
print(jn.collect())

In [ ]:
# 5️⃣ From transactions.txt, identify the most popular product category by: Joining with products.txt
# Counting the number of transactions for each category.

m_5 = t_rdd.map(lambda f: f.split(",")).map(lambda g: (g[1], 1))
mr = m_5.reduceByKey(lambda b, c: b + c)
pp_r = p_rdd.map(lambda o: o.split(",")).map(lambda c: (c[0], c[2]))
join_5 = mr.join(pp_r).sortBy(lambda x: -x[1][0])
print(join_5.take(5))

In [ ]:

# 6️⃣ From transactions.txt, find the average transaction amount per product.

m_6 = t_rdd.map(lambda x: x.split(',')).map(lambda y: (y[1], (int(float(y[2])), 1)))
re6 = m_6.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
final_res = re6.mapValues(lambda x: x[0] / x[1])
print(final_res.collect())

In [ ]:
# 7️⃣ Identify users who have made more than 5 transactions.

us = t_rdd.map(lambda v: v.split(",")).map(lambda f: (f[0], 1)).reduceByKey(lambda x, c: x + c)
fil = us.filter(lambda x: x[1] > 5)
print(fil.collect())

In [ ]:

# 8️⃣ Given a list of transactions, find the highest transaction amount for each user.

uss = t_rdd.map(lambda f: f.split(",")).map(lambda g: (g[0], int(float(g[2]))))
rt = uss.reduceByKey(lambda d, s: max(d, s))
print(rt.collect())

In [ ]:
# 9️⃣ From transactions.txt, count the number of transactions per month.

dff = t_rdd.map(lambda f: f.split(",")).map(lambda g: (g[3].split("-")[1], 1))
ree = dff.reduceByKey(lambda v, g: v + g)
print(ree.collect())


In [ ]:

# 🔟 For each city in users.txt, calculate:

user_city = u_rdd.map(lambda x: x.split(",")).map(lambda y: (y[0], y[3]))
user_amount = t_rdd.map(lambda x: x.split(",")).map(lambda y: (y[0], int(float(y[2]))))


In [ ]:
#Joining 
jn = user_city.join(user_amount)
print(jn.collect())
ct = jn.map(lambda x: (x[1][0], 1)).reduceByKey(lambda z, v: z + v)
mp = jn.map(lambda x : (x[1][0],(x[1][1],1))).reduceByKey(lambda x ,y : (x[0] + y[0],x[1] + y[1])).mapValues(lambda x: x[0] / x[1])
print(f"Averege : {mp.collect()}")
print(f"The count of user is {ct.collect()}")